In [4]:
import pandas as pd
import numpy as np
import networkx as nx
import openpyxl
from openpyxl import Workbook
import networkx as nx
from openpyxl import load_workbook
import matplotlib.pyplot as plt
from openpyxl.utils import get_column_letter
from random import randint
from mysql.connector import connect
import networkx as nx
import pandas as pd
import community as community_louvain

In [5]:
data = pd.read_csv("./holder_edge/result.csv",names=['from_addr','to_addr'],sep="|")
data.head()

,from_addr,to_addr
0,0xafaa71efde96621839214ed8f763f7eb8936be0a,0xb12f8f13ba8e5064c4dd63cee507fd5fa2b7782c
1,0xbf5928c1d26d956805bbbfb6986b445b737618dd,0x9ffae5caa64c33549122e5a68140abe0c4192a9a
2,0x2a613275bb355407bd90ea3e4e5ed0fcdb7107e8,0xd77579390405d1e2a106f777104022a950c5d7c9
3,0x625f880282786d90769a657d5cc2e3b34c4e2147,0x5da2690b20cdafb31eee5ccacfd59009326e9e28
4,0xa75fee88e663bcd2f8ebd8203877cb963d9d37db,0x85c484dbce607ab15d4be2b24373b3836e9c6437


In [6]:

# 删除热点
def select_hot_address(data, field1, field2, n):
    from_to_cnt = data.groupby([field1]).agg({field2:'nunique'}).add_suffix("_cnt")
    source_from_to_cnt = from_to_cnt.sort_values(by=[field2+'_cnt'], ascending=False)
    hot_address_list = source_from_to_cnt.index.tolist()[:n]
    return hot_address_list

# 图算法，先用连通图，然后剔热点，然后用louvain算法
def recursive_louvain(graph, cluster_size):# 参数分别为进入的df和允许的最大cluster的节点数
    sub_graph_edge = []
    node_info = pd.DataFrame()
    g = nx.Graph()
    g.add_edges_from(graph[['from_addr','to_addr']].values)
# 连通图算法
    for index,sub_g in enumerate(nx.connected_components(g)):
        sub_g = g.subgraph(sub_g)
        g_node = sub_g.nodes(sub_g)
        g_node = [g_per[0] for g_per in g_node]
        g_node_tag = [index]*len(g_node)
        node_tag_df = pd.DataFrame()
        node_tag_df['node_addr'] = g_node
        node_tag_df['cluster_id'] = g_node_tag
        node_info = pd.concat([node_info, node_tag_df])

        edg_df = pd.DataFrame(list(sub_g.edges()))
        edg_df.columns = ['from_addr','to_addr']
        edg_df['node_cnt'] = len(sub_g.nodes())
        edg_df['tag'] = index
        sub_graph_edge.append(edg_df)

    sub_graph_edge = pd.concat(sub_graph_edge,ignore_index=True)# 存储连通子图
    node_infos_size=node_info.groupby('cluster_id').size().reset_index() # 对结果计算cluster的size
    node_infos_size.columns=['cluster_id','size']
    large_clusters=node_infos_size[node_infos_size['size']>cluster_size]['cluster_id'].to_list()# 筛选大于最大size的数据
    small_clusters=node_infos_size[node_infos_size['size']<=cluster_size]['cluster_id'].to_list()# 筛选小于等于最大size的数据
    samll_node_info_df=node_info[node_info['cluster_id'].isin(small_clusters)]# 将小于等于最大size的编号对应关系保存
    if len(large_clusters)==0:
        result_df= samll_node_info_df
        return result_df
    else:
        result_df = pd.DataFrame()
        for c in large_clusters:# 针对大于最大size的进行下一轮算法
            sug_graph = sub_graph_edge[sub_graph_edge['tag'] == c]
            # 剔除热点
            from_hot_addr_list = select_hot_address(sug_graph, 'from_addr', 'to_addr', 1)
            to_hot_addr_list = select_hot_address(sug_graph, 'to_addr', 'from_addr', 1)
            del_hot_addr_df = sug_graph[~((sug_graph['from_addr'].isin(from_hot_addr_list))|(sug_graph['to_addr'].isin(to_hot_addr_list)))]
            #用louvain算法
            G = nx.Graph()
            G.add_edges_from(del_hot_addr_df[['from_addr', 'to_addr']].values)
            partition = community_louvain.best_partition(G, resolution=0.1)
            node_info_cluster = pd.DataFrame(list(partition.items()), columns=['node_addr', 'cluster_id'])
            # 对新一轮计算出来的cluster_id做上标记，防止与前面编号冲突
            node_info_cluster['cluster_id'] = node_info_cluster['cluster_id'].apply(lambda x: str(x) + "_" + str(c))
            result_df = pd.concat([result_df, node_info_cluster])
        # 存储前面小于等于最大size的数据和后续被重新划分后的数据
        result_df = pd.concat([samll_node_info_df, result_df])
    return result_df

In [7]:
result_df = recursive_louvain(data, 200)

In [8]:
result_df

,node_addr,cluster_id
0,0x0a6a81bfeb4d7a58f61ba9f7b22ef3fc69bf8488,0
1,0xafaa71efde96621839214ed8f763f7eb8936be0a,0
2,0x3506fbe85e19bf025b228ec58f143ba342c3c608,0
3,0xb12f8f13ba8e5064c4dd63cee507fd5fa2b7782c,0
4,0x00d86cf36cfa992e69dcb51f0429eea74f105791,0
...,...,...
184,0xefdeed67c19cda6b078f1640972647ba1376eda4,26_588
185,0x56fc8504455434565110cf7aeadb9594f9ff8469,26_588
186,0xa3ab4c8e2c3d87ec657bb6480755b6e8b1a15892,6_588
0,0x2be1a6516ab37aec1240faff34a2a6c0d3f6429f,0_2772


In [30]:
result_cnt = result_df.groupby(['cluster_id']).agg({'node_addr':'nunique'})
result_cnt = result_cnt.sort_values(by=['node_addr'],ascending=False)
result_cnt = result_cnt[result_cnt['node_addr'] > 5].reset_index()
result_cnt['node_addr'] = result_cnt['node_addr'].astype(str)
result_df['node_addr'] = result_df['node_addr'].astype(str)
result_df
result_df_new = result_df.merge(result_cnt[['cluster_id']],how='inner',on=['cluster_id'])
result_df_new
# result_df_new

,node_addr,cluster_id
0,0x0a6a81bfeb4d7a58f61ba9f7b22ef3fc69bf8488,0
1,0xafaa71efde96621839214ed8f763f7eb8936be0a,0
2,0x3506fbe85e19bf025b228ec58f143ba342c3c608,0
3,0xb12f8f13ba8e5064c4dd63cee507fd5fa2b7782c,0
4,0x00d86cf36cfa992e69dcb51f0429eea74f105791,0
...,...,...
44244,0x2bc2df1c963964f83adc2ab9f76c59a4ad6cb8ed,55_588
44245,0xcc45c36413cc8552df67847569a4858c657ae4a3,55_588
44246,0x8360f7bc3ff9d62f5c53c056a6670e91154a1dbe,55_588
44247,0x16446990e0befde744745f28a2b72c364ba51f54,55_588


In [36]:
result_cnt

,cluster_id,node_addr
0,190_21,405
1,142_21,316
2,384_21,260
3,152_21,245
4,440_21,239
...,...,...
2354,4216,6
2355,2000,6
2356,4229,6
2357,2087,6


In [34]:
# edge_df = []

# 画出每个组的簇图
import networkx as nx
from openpyxl import load_workbook
import matplotlib.pyplot as plt
from openpyxl.utils import get_column_letter
from random import randint
%matplotlib inline
# 画图    
def plot_dg(df,col1,col2,mark):
    DG = nx.from_pandas_edgelist(
        df=df,
        source=col1,
        target=col2,
#         edge_attr=True,
        create_using=nx.DiGraph()
    )
    fig, ax = plt.subplots(figsize=(20, 13))
    fig.patch.set_facecolor('white')
    pos = nx.spring_layout(DG, iterations=10000, seed=1721)
    node_colors = ['red'  for node in DG.nodes()]
#     node_colors = ['red' if node in climate_list else 'blue' for node in DG.nodes()]
    nx.draw(
        G=DG,
        pos=pos,
        arrowsize=15,
        connectionstyle='arc3, rad = 0.1',
        node_color=node_colors,
    #         with_labels=True,
        node_size=50,
        width=1.5
        )
    nx.draw_networkx_edge_labels(
        DG,
        pos=pos,
        edge_labels=nx.get_edge_attributes(DG,'weight')
        )
    plt.savefig('./test6_png/%s.png'%(mark))
    plt.close()
for index,cid in enumerate(result_df_new['cluster_id'].unique().tolist()):
    print("========================%s========================"%(index))
    node_list = result_df_new['node_addr'][result_df_new['cluster_id'] == cid].unique().tolist()
    edge_data = data[data['from_addr'].isin(node_list) & data['to_addr'].isin(node_list)]
    
    
    plot_dg(edge_data[['from_addr','to_addr']],'from_addr','to_addr',cid)
    edge_data.columns = ['source','target']
    edge_data.to_csv("./lens_cluster_info2/%s.csv"%(cid),index=False)
    
#     edge_data['cid'] = cid
#     edge_df.append(edge_data)
# edge_df = pd.concat(edge_df,ignore_index=True)
# edge_df

========================0========================
========================2========================
========================3========================
========================4========================
========================5========================
========================6========================
========================10========================
========================11========================
========================12========================
========================15========================
========================17========================
========================22========================
========================24========================
========================25========================
========================28========================
========================29========================
========================30========================
========================32========================
========================34========================
========================39===========

========================257========================
========================259========================
========================260========================
========================261========================
========================262========================
========================264========================
========================265========================
========================266========================
========================267========================
========================268========================
========================270========================
========================271========================
========================272========================
========================276========================
========================277========================
========================278========================
========================279========================
========================280========================
========================281========================
============

========================504========================
========================505========================
========================506========================
========================509========================
========================510========================
========================511========================
========================512========================
========================516========================
========================517========================
========================518========================
========================519========================
========================521========================
========================522========================
========================526========================
========================528========================
========================532========================
========================533========================
========================536========================
========================538========================
============

========================775========================
========================783========================
========================784========================
========================786========================
========================787========================
========================791========================
========================792========================
========================793========================
========================794========================
========================801========================
========================809========================
========================812========================
========================813========================
========================814========================
========================816========================
========================818========================
========================820========================
========================821========================
========================822========================
============

========================1147========================
========================1148========================
========================1149========================
========================1151========================
========================1153========================
========================1154========================
========================1155========================
========================1157========================
========================1159========================
========================1161========================
========================1164========================
========================1165========================
========================1166========================
========================1167========================
========================1169========================
========================1170========================
========================1171========================
========================1174========================
========================1177==================

...

========================474_21========================
========================458_21========================
========================136_21========================
========================299_21========================
========================138_21========================
========================266_21========================
========================560_21========================
========================273_21========================
========================147_21========================
========================141_21========================
========================570_21========================
========================372_21========================
========================151_21========================
========================215_21========================
========================212_21========================
========================216_21========================
========================279_21========================
========================169_21========================
==========

========================493_21========================
========================502_21========================
========================290_21========================
========================326_21========================
========================507_21========================
========================18_21========================
========================514_21========================
========================231_21========================
========================416_21========================
========================526_21========================
========================516_21========================
========================530_21========================
========================371_21========================
========================590_21========================
========================537_21========================
========================154_21========================
========================349_21========================
========================145_21========================
===========

========================268_21========================
========================269_21========================
========================270_21========================
========================127_21========================
========================271_21========================
========================274_21========================
========================504_21========================
========================282_21========================
========================51_21========================
========================285_21========================
========================313_21========================
========================137_21========================
========================292_21========================
========================300_21========================
========================553_21========================
========================305_21========================
========================307_21========================
========================309_21========================
===========

========================32_26========================
========================110_26========================
========================87_26========================
========================98_26========================
========================66_26========================
========================37_27========================
========================6_27========================
========================26_27========================
========================1_31========================
========================3_31========================
========================6_31========================
========================22_31========================
========================28_31========================
========================32_31========================
========================39_31========================
========================33_31========================
========================94_35========================
========================4_35========================
========================119_35==

========================184_145========================
========================106_145========================
========================130_145========================
========================183_145========================
========================1_185========================
========================10_185========================
========================8_185========================
========================12_185========================
========================48_185========================
========================42_185========================
========================21_185========================
========================50_185========================
========================68_185========================
========================33_185========================
========================19_185========================
========================54_185========================
========================57_185========================
========================58_185========================
========

In [16]:
edge_data

,from_addr,to_addr
0,0xafaa71efde96621839214ed8f763f7eb8936be0a,0xb12f8f13ba8e5064c4dd63cee507fd5fa2b7782c
1833,0xb12f8f13ba8e5064c4dd63cee507fd5fa2b7782c,0xd89ce37f247290f4b404db7415deed1a0cf163b1
3595,0xb12f8f13ba8e5064c4dd63cee507fd5fa2b7782c,0xafaa71efde96621839214ed8f763f7eb8936be0a
4287,0xd89ce37f247290f4b404db7415deed1a0cf163b1,0xafaa71efde96621839214ed8f763f7eb8936be0a
23580,0x0a6a81bfeb4d7a58f61ba9f7b22ef3fc69bf8488,0xafaa71efde96621839214ed8f763f7eb8936be0a
25385,0x00d86cf36cfa992e69dcb51f0429eea74f105791,0xafaa71efde96621839214ed8f763f7eb8936be0a
47805,0x00d86cf36cfa992e69dcb51f0429eea74f105791,0xb12f8f13ba8e5064c4dd63cee507fd5fa2b7782c
50694,0x3506fbe85e19bf025b228ec58f143ba342c3c608,0xafaa71efde96621839214ed8f763f7eb8936be0a
56032,0x3506fbe85e19bf025b228ec58f143ba342c3c608,0x0a6a81bfeb4d7a58f61ba9f7b22ef3fc69bf8488
57366,0x00d86cf36cfa992e69dcb51f0429eea74f105791,0x3506fbe85e19bf025b228ec58f143ba342c3c608


In [ ]:
DG = nx.from_pandas_edgelist(
    df=edge_data,
    source='from_addr',
    target='to_addr',
    edge_attr=True,
    create_using=nx.DiGraph()
)

In [ ]:

# 画出每个组的簇图
import networkx as nx
from openpyxl import load_workbook
import matplotlib.pyplot as plt
from openpyxl.utils import get_column_letter
from random import randint
%matplotlib inline
# 画图    
def plot_dg(df,col1,col2,mark):
    DG = nx.from_pandas_edgelist(
        df=df,
        source=col1,
        target=col2,
        edge_attr=True,
        create_using=nx.DiGraph()
    )
    fig, ax = plt.subplots(figsize=(20, 13))
    fig.patch.set_facecolor('white')
    pos = nx.spring_layout(DG, iterations=10000, seed=1721)
    node_colors = ['red' if node in climate_list else 'blue' for node in DG.nodes()]
    nx.draw(
        G=DG,
        pos=pos,
        arrowsize=15,
        connectionstyle='arc3, rad = 0.1',
        node_color=node_colors,
    #         with_labels=True,
        node_size=50,
        width=1.5
        )
    nx.draw_networkx_edge_labels(
        DG,
        pos=pos,
        edge_labels=nx.get_edge_attributes(DG,'weight')
        )
    plt.savefig('./test5_png/%s.png'%(mark))
    plt.close()
# 将数据写出存储，并将图片存储写出
with pd.ExcelWriter('data1.xlsx') as writer:
    sybil_address_df.to_excel(writer, sheet_name='sheet1',index=False)
    cluster_df_select.to_excel(writer, sheet_name='sheet2',index=False)
    for cid in connected_graph_df['cluster_id'].unique():
        print(cid)
        plot_dg(connected_graph_df[connected_graph_df['cluster_id'] == cid],'from_addr','to_addr',cid)
print('done')